In [2]:
#import the required modules
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import matplotlib.pyplot as plt
from datetime import datetime, date

In [3]:
def calculate_age(born):
    born = datetime.strptime(born, "%d/%m/%Y").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [4]:
df = pd.read_csv("~/Documents/zarathustra/inferenceage.csv")
df = df.rename(columns={"CD_NGR":"cd_ngr"})
for i in np.arange(len(df)):
    df['Esiti'][i] = str(df['Esiti'][i])
print(df.Esiti.value_counts())

print(df)
print(df.Esiti.value_counts())
df = df.drop(columns={'Classe affidamento','codice SRE','Unnamed: 7'})
columnsTitles=["cd_ngr","DT_NASC","fg_mail","fg_tel","Esiti"]
df=df.reindex(columns=columnsTitles)
dftest = pd.read_csv("~/Documents/zarathustra/massivo_test_jan.csv")
df3=pd.merge(dftest,df,on="cd_ngr")
print(df3)
print(df3.columns)
df3 = df3.drop(columns=['cd_rae_coint_principale','dt_ini_fase_x','dt_fine_fase_x','dt_ini_fase_y', 'dt_fine_fase_y'])
df3 = df3.rename(columns={'fg_tel':'phone','fg_mail':'email','DT_NASC':'birth'})
print(df3.columns)
wrongtest = []
for i in np.arange(len(df3)):
    if df3['Esiti'][i] == "-":
        wrongtest.append(i)
df3 = df3.drop(wrongtest)
print(df3.Esiti.value_counts())
df3[['Esiti']] = df3['Esiti'].map({'Esazione negativa (nulla da segnalare)':1,
                                'Cliente promette pagamento ma non mantiene':1,
                                'Pagamento Eseguito (anche parziale)':0,
                                  'Cliente rifiuta il pagamento': 1,
                                  "Cliente deceduto": 1,
                                   "Cliente contesta il debito": 1,
                                   "Procedure concorsuali": 1,
                                   "Cliente riferito all'estero": 1,
                                   "Cliente contattato (Comunicato nuovo recapito)":1,
                                   "La SRE chiede ulteriore assegnazione dell'NDC":1,
                                   "Cliente riferisce difficoltà economiche": 1,
                                   "Cliente riferisce accordi pregressi":1,
                                   "Cliente assente (recapiti validi ma non reperito)": 1,
                                   "Cliente irreperibile (recapiti assenti o non validi)":1,
                                  "Piano di rientro":0,
                                  "Saldo e stralcio":0,
                                  "Pagamento in transito (da contabilizzare)":0,
                                  "Transazione conclusa":0,
                                  })

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Cliente assente (recapiti validi ma non reperito)       2484
Cliente irreperibile (recapiti assenti o non validi)    2364
Cliente rifiuta il pagamento                            1397
Cliente promette pagamento ma non mantiene              1170
Pagamento Eseguito (anche parziale)                      733
La SRE chiede ulteriore assegnazione dell'NDC            464
Esazione negativa (nulla da segnalare)                   451
Cliente riferisce difficoltà economiche                  256
Cliente contattato (Comunicato nuovo recapito)           116
Cliente riferisce accordi pregressi                      116
Pagamento in transito (da contabilizzare)                 88
-                                                         52
Cliente deceduto                                          48
Saldo e stralcio                                          43
Transazione conclusa                                      30
Piano di rientro                                          24
Cliente contesta il debi

In [5]:
print(df3.Esiti.isna().value_counts())

False    5601
Name: Esiti, dtype: int64


In [7]:
print(len(df3))
print(df3)
df3 = df3.reset_index()
df3 = pd.DataFrame(df3)
wrong3 = []
for i in np.arange(len(df3)):
    if df3['birth'][i] == "nan" or df3['birth'][i] == "NaN":
        wrong3.append(i)
df3 = df3.drop(wrong3)
print(len(df3))
df3[['birth']] = df3['birth'].astype('str')
df3['birth'] = df3['birth'].apply(calculate_age)
print(df3.birth.value_counts())

5601
      index     cd_ngr ty_cliente  nr_cointestatari_secondari  \
0         0   47434911        COI                           1   
1         1   14867841        SPF                           0   
2         2   27846321        SPF                           0   
3         3   25764078        SPF                           0   
4         4   68036583        SPF                           0   
...     ...        ...        ...                         ...   
5596   5629  142030919        SPF                           0   
5597   5630  213610583        SPF                           0   
5598   5631  248751671        SDC                           0   
5599   5632  264575831        SPF                           0   
5600   5633  307560503        SRP                           0   

      cd_sae_coint_principale tx_sigla_prv_coint_principale  nr_rapp_forb_ini  \
0                         600                            CT                 0   
1                         600                       

In [8]:
df3[['ty_cliente']] = df3['ty_cliente'].map({'SPF':1, 'SDC':2, 'SRP':3,'SGP':4, 'SDF':5, 'COI':6,'PUB':7})
df3['tx_sigla_prv_coint_principale'] = df3['tx_sigla_prv_coint_principale'].astype(str)
from sklearn.preprocessing import LabelEncoder

lb_make2 = LabelEncoder()
df3["tx_sigla_prv_coint_principale"] = lb_make2.fit_transform(df3["tx_sigla_prv_coint_principale"])
print(df3['tx_sigla_prv_coint_principale'].head())

1     81
2     28
3     47
4     65
5    105
Name: tx_sigla_prv_coint_principale, dtype: int64


In [9]:
df3 = df3.drop(columns=['level_0'])
df3 = df3.reset_index()
df3 = pd.DataFrame(df3)
nan1 = []
for i in np.arange(len(df3)):
    if df3['im_saldo_ini'][i] == 0:
        nan1.append(i)
    else:
        continue 
df3 = df3.drop(nan1)
df3['im_mov_av_t24'] = df3['im_mov_av_t24']/df3['im_saldo_ini']
df3['im_mov_av_t23'] = df3['im_mov_av_t23']/df3['im_saldo_ini']
df3['im_mov_av_t22'] = df3['im_mov_av_t22']/df3['im_saldo_ini']
df3['im_mov_av_t21'] = df3['im_mov_av_t21']/df3['im_saldo_ini']
df3['im_mov_av_t20'] = df3['im_mov_av_t20']/df3['im_saldo_ini']
df3['im_mov_av_t19'] = df3['im_mov_av_t19']/df3['im_saldo_ini']
df3['im_mov_av_t18'] = df3['im_mov_av_t18']/df3['im_saldo_ini']
df3['im_mov_av_t17'] = df3['im_mov_av_t17']/df3['im_saldo_ini']
df3['im_mov_av_t16'] = df3['im_mov_av_t16']/df3['im_saldo_ini']
df3['im_mov_av_t15'] = df3['im_mov_av_t15']/df3['im_saldo_ini']
df3['im_mov_av_t14'] = df3['im_mov_av_t14']/df3['im_saldo_ini']
df3['im_mov_av_t13'] = df3['im_mov_av_t13']/df3['im_saldo_ini']
df3['im_mov_av_t12'] = df3['im_mov_av_t12']/df3['im_saldo_ini']
df3['im_mov_av_t11'] = df3['im_mov_av_t11']/df3['im_saldo_ini']
df3['im_mov_av_t10'] = df3['im_mov_av_t10']/df3['im_saldo_ini']
df3['im_mov_av_t9'] = df3['im_mov_av_t9']/df3['im_saldo_ini']
df3['im_mov_av_t8'] = df3['im_mov_av_t8']/df3['im_saldo_ini']
df3['im_mov_av_t7'] = df3['im_mov_av_t7']/df3['im_saldo_ini']
df3['im_mov_av_t6'] = df3['im_mov_av_t6']/df3['im_saldo_ini']
df3['im_mov_av_t5'] = df3['im_mov_av_t5']/df3['im_saldo_ini']
df3['im_mov_av_t4'] = df3['im_mov_av_t4']/df3['im_saldo_ini']
df3['im_mov_av_t3'] = df3['im_mov_av_t3']/df3['im_saldo_ini']
df3['im_mov_av_t2'] = df3['im_mov_av_t2']/df3['im_saldo_ini']
df3['im_mov_av_t1'] = df3['im_mov_av_t1']/df3['im_saldo_ini']

In [10]:
inference =df3.drop(columns=['level_0','cd_ngr','Esiti','index','fg_cli_forb_ini','fg_cli_forb_sconf30_ini'])

In [11]:
df0129 = pd.read_csv("~/Documents/zarathustra/massivo_training.csv")
dfage = pd.read_csv("~/Documents/zarathustra/massivoage.csv")

print(df0129.head(n=20))
print(df0129.columns[:100])
print(len(df0129))
print(dfage.columns)
print(dfage.head(30))
print(len(dfage))

      cd_ngr ty_cliente  cd_ngr_coint_principale  nr_cointestatari_secondari  \
0   10160304        SPF                 10160304                           0   
1   14171016        SPF                 14171016                           0   
2   26292648        SPF                 26292648                           0   
3   25574592        SPF                 25574592                           0   
4   33139080        SPF                 33139080                           0   
5    2427118        SPF                  2427118                           0   
6    5254438        SDC                  5254438                           0   
7    5891206        SPF                  5891206                           0   
8   44260896        SPF                 44260896                           0   
9   54440592        COI                 32468594                           1   
10  16591102        SPF                 16591102                           0   
11  54675000        COI                 

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
print(len(df0129))
data = pd.merge(df0129,dfage,on='cd_ngr')
data = data.drop_duplicates(subset='cd_ngr')
print(len(data))

58059
58058


In [13]:
print(data.columns)

Index(['cd_ngr', 'ty_cliente', 'cd_ngr_coint_principale',
       'nr_cointestatari_secondari', 'cd_ngr_coint_coll_1',
       'cd_ngr_coint_coll_2', 'cd_ngr_coint_coll_3',
       'cd_piva_coint_principale', 'cd_fiscale_coint_principale',
       'cd_rae_coint_principale',
       ...
       'im_acc_cassa_t7', 'im_acc_cassa_t6', 'im_acc_cassa_t5',
       'im_acc_cassa_t4', 'im_acc_cassa_t3', 'im_acc_cassa_t2',
       'im_acc_cassa_t1', 'birth', 'email', 'phone'],
      dtype='object', length=118)


In [14]:
wrong1 = []
for i in np.arange(len(data)):
    if data['birth'][i] == "00/01/1900":
        wrong1.append(i)
    if data['birth'][i] == "12/29/1899":
        wrong1.append(i)
data = data.drop(wrong1)
print(len(data))

45544


In [15]:
data = data.reset_index()
data = pd.DataFrame(data)
wrong2 = []
for i in np.arange(len(data)):
    if data['birth'][i] == "-":
        wrong2.append(i)
data = data.drop(wrong2)
print(len(data))

41859


In [16]:
print(len(data))
data = data.reset_index()
data = pd.DataFrame(data)
nan = []
count = 0
for i in np.arange(len(data)):
    if data['im_saldo_ini'][i] == 0:
        nan.append(i)
        count = count + 1 
    else:
        continue 
print(nan)
data = data.drop(nan)
print(len(data['response']))

41859
[401, 677, 1509, 1551, 1786, 2015, 2106, 3092, 3540, 5766, 6354, 7876, 8172, 8745, 8871, 9581, 10150, 10250, 11825, 12983, 13441, 13504, 13666, 14288, 14543, 14956, 15469, 15988, 16378, 16997, 17547, 17622, 17733, 19175, 19498, 19652, 19947, 20798, 21407, 22074, 23008, 23012, 23244, 23782, 24055, 25363, 27461, 27468, 27883, 29037, 29873, 30410, 31009, 31197, 31740, 32583, 32609, 32708, 33435, 34045, 35062, 35311, 36680, 36991, 37367, 37715, 37796, 38604, 39076, 39980, 40087, 40307, 40353, 40363, 41494]
41784


In [17]:
data = data.drop(columns=['cd_ngr_coint_principale','cd_ngr_coint_coll_1','cd_ngr_coint_coll_2','cd_ngr_coint_coll_3','cd_rae_coint_principale','cd_piva_coint_principale','cd_fiscale_coint_principale','cd_piva_coint_coll_1','cd_fiscale_coint_coll_1','cd_piva_coint_coll_2','cd_fiscale_coint_coll_2','cd_piva_coint_coll_3','cd_fiscale_coint_coll_3'])
data = data.drop(columns=['dt_ini_fase_x','dt_fine_fase_x'])
#data = data.drop(data.iloc[:,15:25],axis=1)
print(data.head())
print(len(data['response']))

   level_0  index    cd_ngr ty_cliente  nr_cointestatari_secondari  \
0        0      0  10160304        SPF                           0   
1        1      1  14171016        SPF                           0   
2        2      2  26292648        SPF                           0   
3        3      3  25574592        SPF                           0   
4        4      4  33139080        SPF                           0   

   cd_sae_coint_principale tx_sigla_prv_coint_principale  nr_rapp_forb_ini  \
0                      614                            ME                 0   
1                      615                            RM                 0   
2                      600                            GR                 0   
3                      600                            PA                 0   
4                      600                            PT                 0   

   nr_rapp_forb_sconf30_ini  im_saldo_ini  ...  im_acc_cassa_t7  \
0                         0      11366.74  

In [18]:
data[['ty_cliente']] = data['ty_cliente'].map({'SPF':1, 'SDC':2, 'SRP':3,'SGP':4, 'SDF':5, 'COI':6,'PUB':7})
data['tx_sigla_prv_coint_principale'] = data['tx_sigla_prv_coint_principale'].astype(str)
print(data.ty_cliente.unique())
print(data.tx_sigla_prv_coint_principale.unique())
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
data["tx_sigla_prv_coint_principale"] = lb_make.fit_transform(data["tx_sigla_prv_coint_principale"])
print(data['tx_sigla_prv_coint_principale'].head())
print(len(data['response']))

[1 6 3 2 4 5 7]
['ME' 'RM' 'GR' 'PA' 'PT' 'LT' 'CT' 'AP' 'AG' 'CS' 'CE' 'nan' 'MI' 'PG'
 'PI' 'LI' 'TP' 'FR' 'TO' 'LU' 'FI' 'SR' 'BA' 'SA' 'AR' 'TA' 'RG' 'MS'
 'SI' 'CH' 'LE' 'MT' 'TE' 'BT' 'CA' 'NO' 'LC' 'VC' 'RC' 'TR' 'VT' 'CZ'
 'PU' 'PV' 'FC' 'FM' 'VI' 'CR' 'PN' 'GE' 'AN' 'EN' 'RI' 'CL' 'MC' 'AQ'
 'MN' 'FG' 'BG' 'TV' 'CB' 'PZ' 'BS' 'CN' 'MB' 'VE' 'PD' 'VV' 'VA' 'BO'
 'SV' 'PE' 'MO' 'VR' 'PR' 'PO' 'CO' 'SP' 'IS' 'UD' 'OR' 'SS' 'BR' 'AV'
 'AO' 'GO' 'RA' 'AL' 'TS' 'RN' 'VB' 'RO' 'RE' 'SU' 'LO' 'TN' 'IM' 'BZ'
 'FE' 'BL' 'BN' 'NU' 'AT' 'BI' 'KR' 'PC' 'SO']
0    51
1    78
2    39
3    60
4    69
Name: tx_sigla_prv_coint_principale, dtype: int64
41784


In [19]:
data = data.drop(data.iloc[:,15:25],axis=1)
data = data.drop(columns=['dt_fine_fase_y','dt_ini_fase_y'])

In [20]:
data['im_mov_av_t24'] = data['im_mov_av_t24']/data['im_saldo_ini']
data['im_mov_av_t23'] = data['im_mov_av_t23']/data['im_saldo_ini']
data['im_mov_av_t22'] = data['im_mov_av_t22']/data['im_saldo_ini']
data['im_mov_av_t21'] = data['im_mov_av_t21']/data['im_saldo_ini']
data['im_mov_av_t20'] = data['im_mov_av_t20']/data['im_saldo_ini']
data['im_mov_av_t19'] = data['im_mov_av_t19']/data['im_saldo_ini']
data['im_mov_av_t18'] = data['im_mov_av_t18']/data['im_saldo_ini']
data['im_mov_av_t17'] = data['im_mov_av_t17']/data['im_saldo_ini']
data['im_mov_av_t16'] = data['im_mov_av_t16']/data['im_saldo_ini']
data['im_mov_av_t15'] = data['im_mov_av_t15']/data['im_saldo_ini']
data['im_mov_av_t14'] = data['im_mov_av_t14']/data['im_saldo_ini']
data['im_mov_av_t13'] = data['im_mov_av_t13']/data['im_saldo_ini']
data['im_mov_av_t12'] = data['im_mov_av_t12']/data['im_saldo_ini']
data['im_mov_av_t11'] = data['im_mov_av_t11']/data['im_saldo_ini']
data['im_mov_av_t10'] = data['im_mov_av_t10']/data['im_saldo_ini']
data['im_mov_av_t9'] = data['im_mov_av_t9']/data['im_saldo_ini']
data['im_mov_av_t8'] = data['im_mov_av_t8']/data['im_saldo_ini']
data['im_mov_av_t7'] = data['im_mov_av_t7']/data['im_saldo_ini']
data['im_mov_av_t6'] = data['im_mov_av_t6']/data['im_saldo_ini']
data['im_mov_av_t5'] = data['im_mov_av_t5']/data['im_saldo_ini']
data['im_mov_av_t4'] = data['im_mov_av_t4']/data['im_saldo_ini']
data['im_mov_av_t3'] = data['im_mov_av_t3']/data['im_saldo_ini']
data['im_mov_av_t2'] = data['im_mov_av_t2']/data['im_saldo_ini']
data['im_mov_av_t1'] = data['im_mov_av_t1']/data['im_saldo_ini']
print(data.head())
print(len(data['response']))
data = data.sample(frac=1)

   level_0  index    cd_ngr  ty_cliente  nr_cointestatari_secondari  \
0        0      0  10160304           1                           0   
1        1      1  14171016           1                           0   
2        2      2  26292648           1                           0   
3        3      3  25574592           1                           0   
4        4      4  33139080           1                           0   

   cd_sae_coint_principale  tx_sigla_prv_coint_principale  nr_rapp_forb_ini  \
0                      614                             51                 0   
1                      615                             78                 0   
2                      600                             39                 0   
3                      600                             60                 0   
4                      600                             69                 0   

   nr_rapp_forb_sconf30_ini  im_saldo_ini  ...  im_acc_cassa_t7  \
0                         0    

In [21]:
data=data.dropna(axis=0)
print(len(data))
print(data)

40761
       level_0  index     cd_ngr  ty_cliente  nr_cointestatari_secondari  \
162        171    217  119137375           1                           0   
29042    31599  40149  162440149           1                           0   
1435      1557   1913  120541260           1                           0   
2048      2220   2709  182498434           1                           0   
10586    11538  14517  168438973           1                           0   
...        ...    ...        ...         ...                         ...   
36500    39732  50637  188007159           1                           0   
25729    28000  35516  140885511           1                           0   
14408    15710  19793  263036920           1                           0   
20208    21987  27835  205058720           1                           0   
23890    26005  32940  247722689           1                           0   

       cd_sae_coint_principale  tx_sigla_prv_coint_principale  \
162             

In [22]:
data['birth'] = data['birth'].apply(calculate_age)
print(data)

       level_0  index     cd_ngr  ty_cliente  nr_cointestatari_secondari  \
162        171    217  119137375           1                           0   
29042    31599  40149  162440149           1                           0   
1435      1557   1913  120541260           1                           0   
2048      2220   2709  182498434           1                           0   
10586    11538  14517  168438973           1                           0   
...        ...    ...        ...         ...                         ...   
36500    39732  50637  188007159           1                           0   
25729    28000  35516  140885511           1                           0   
14408    15710  19793  263036920           1                           0   
20208    21987  27835  205058720           1                           0   
23890    26005  32940  247722689           1                           0   

       cd_sae_coint_principale  tx_sigla_prv_coint_principale  \
162                   

In [23]:
data = data.astype({'email': 'int32','phone':'int32'})
print(data.columns)

Index(['level_0', 'index', 'cd_ngr', 'ty_cliente',
       'nr_cointestatari_secondari', 'cd_sae_coint_principale',
       'tx_sigla_prv_coint_principale', 'nr_rapp_forb_ini',
       'nr_rapp_forb_sconf30_ini', 'im_saldo_ini', 'im_int_mora_ini',
       'im_saldo_netto_mora_ini', 'im_gar_reali_ini', 'im_gar_person_ini',
       'im_gar_altre_ini', 'fg_cli_forb_fine', 'fg_cli_forb_sconf30_fine',
       'response', 'im_mov_av_t24', 'im_mov_av_t23', 'im_mov_av_t22',
       'im_mov_av_t21', 'im_mov_av_t20', 'im_mov_av_t19', 'im_mov_av_t18',
       'im_mov_av_t17', 'im_mov_av_t16', 'im_mov_av_t15', 'im_mov_av_t14',
       'im_mov_av_t13', 'im_mov_av_t12', 'im_mov_av_t11', 'im_mov_av_t10',
       'im_mov_av_t9', 'im_mov_av_t8', 'im_mov_av_t7', 'im_mov_av_t6',
       'im_mov_av_t5', 'im_mov_av_t4', 'im_mov_av_t3', 'im_mov_av_t2',
       'im_mov_av_t1', 'im_util_cassa_t24', 'im_util_cassa_t23',
       'im_util_cassa_t22', 'im_util_cassa_t21', 'im_util_cassa_t20',
       'im_util_cassa_t19', 'im_u

In [24]:
data = data.drop(columns=['level_0'])
data = data.reset_index()
data = pd.DataFrame(data)
print(data.columns)

Index(['level_0', 'index', 'cd_ngr', 'ty_cliente',
       'nr_cointestatari_secondari', 'cd_sae_coint_principale',
       'tx_sigla_prv_coint_principale', 'nr_rapp_forb_ini',
       'nr_rapp_forb_sconf30_ini', 'im_saldo_ini', 'im_int_mora_ini',
       'im_saldo_netto_mora_ini', 'im_gar_reali_ini', 'im_gar_person_ini',
       'im_gar_altre_ini', 'fg_cli_forb_fine', 'fg_cli_forb_sconf30_fine',
       'response', 'im_mov_av_t24', 'im_mov_av_t23', 'im_mov_av_t22',
       'im_mov_av_t21', 'im_mov_av_t20', 'im_mov_av_t19', 'im_mov_av_t18',
       'im_mov_av_t17', 'im_mov_av_t16', 'im_mov_av_t15', 'im_mov_av_t14',
       'im_mov_av_t13', 'im_mov_av_t12', 'im_mov_av_t11', 'im_mov_av_t10',
       'im_mov_av_t9', 'im_mov_av_t8', 'im_mov_av_t7', 'im_mov_av_t6',
       'im_mov_av_t5', 'im_mov_av_t4', 'im_mov_av_t3', 'im_mov_av_t2',
       'im_mov_av_t1', 'im_util_cassa_t24', 'im_util_cassa_t23',
       'im_util_cassa_t22', 'im_util_cassa_t21', 'im_util_cassa_t20',
       'im_util_cassa_t19', 'im_u

In [25]:
X = data.drop(columns=['response','cd_ngr','level_0','index','fg_cli_forb_fine', 'fg_cli_forb_sconf30_fine'])
Y = data[['response']]
print(Y)
Y.loc[Y.response > 0] = 1
Y.loc[Y.response <= 0]= 0
print(Y)

       response
0         38.63
1         32.40
2     -18478.36
3     -27665.66
4         96.91
...         ...
40756     23.84
40757    455.15
40758     35.28
40759    165.67
40760 -37962.19

[40761 rows x 1 columns]
       response
0           1.0
1           1.0
2           0.0
3           0.0
4           1.0
...         ...
40756       1.0
40757       1.0
40758       1.0
40759       1.0
40760       0.0

[40761 rows x 1 columns]


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
print(Y.sum(),len(Y))

response    32460.0
dtype: float64 40761


In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [28]:
print(pd.DataFrame(x_train).columns)
print(pd.DataFrame(x_test).columns)

Index(['ty_cliente', 'nr_cointestatari_secondari', 'cd_sae_coint_principale',
       'tx_sigla_prv_coint_principale', 'nr_rapp_forb_ini',
       'nr_rapp_forb_sconf30_ini', 'im_saldo_ini', 'im_int_mora_ini',
       'im_saldo_netto_mora_ini', 'im_gar_reali_ini', 'im_gar_person_ini',
       'im_gar_altre_ini', 'im_mov_av_t24', 'im_mov_av_t23', 'im_mov_av_t22',
       'im_mov_av_t21', 'im_mov_av_t20', 'im_mov_av_t19', 'im_mov_av_t18',
       'im_mov_av_t17', 'im_mov_av_t16', 'im_mov_av_t15', 'im_mov_av_t14',
       'im_mov_av_t13', 'im_mov_av_t12', 'im_mov_av_t11', 'im_mov_av_t10',
       'im_mov_av_t9', 'im_mov_av_t8', 'im_mov_av_t7', 'im_mov_av_t6',
       'im_mov_av_t5', 'im_mov_av_t4', 'im_mov_av_t3', 'im_mov_av_t2',
       'im_mov_av_t1', 'im_util_cassa_t24', 'im_util_cassa_t23',
       'im_util_cassa_t22', 'im_util_cassa_t21', 'im_util_cassa_t20',
       'im_util_cassa_t19', 'im_util_cassa_t18', 'im_util_cassa_t17',
       'im_util_cassa_t16', 'im_util_cassa_t15', 'im_util_cassa_t14

In [29]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
rf = XGBClassifier(n_estimators=212,max_depth = 13)
rf.fit(x_train, y_train)
print(rf.score(x_train,y_train))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9911371442590775


In [30]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(x_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.50%


In [31]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.92      0.75      0.83      1687
         1.0       0.94      0.98      0.96      6466

    accuracy                           0.93      8153
   macro avg       0.93      0.87      0.89      8153
weighted avg       0.93      0.93      0.93      8153

[[1273  414]
 [ 116 6350]]


In [ ]:
from xgboost import plot_importance
ax = plot_importance(rf)
fig = ax.figure
fig.set_size_inches(100, 25)

In [32]:
inference['birth'] = inference['birth'].astype('int32')
y_inference_pred = rf.predict(inference)
print(y_inference_pred)

[1. 1. 1. ... 0. 0. 1.]


In [33]:
df3['model_results'] = y_inference_pred
results2211 = df3[['cd_ngr','model_results','Esiti']]
results2211 = results2211.reset_index()
results2211 = pd.DataFrame(results2211)

In [34]:
results2211 = results2211.drop(columns=['index'])
count = 0
for i in np.arange(len(results2211)):
    if results2211['model_results'][i] == results2211['Esiti'][i]:
        count += 1
print(count)
print(confusion_matrix(results2211[['Esiti']],y_inference_pred,))
print(classification_report(results2211[['Esiti']],y_inference_pred))

2897
[[  48  140]
 [ 328 2849]]
              precision    recall  f1-score   support

           0       0.13      0.26      0.17       188
           1       0.95      0.90      0.92      3177

    accuracy                           0.86      3365
   macro avg       0.54      0.58      0.55      3365
weighted avg       0.91      0.86      0.88      3365



In [35]:
results2211.to_csv('~/Documents/zarathustra/results2211.csv')